# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
M. Ravet  ->  M. Ravet  |  ['M. Ravet']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


D. Semenov  ->  D. Semenov  |  ['D. Semenov']
T. Henning  ->  T. Henning  |  ['T. Henning']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Arxiv has 99 new papers today
          11 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/11 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.25314


extracting tarball to tmp_2509.25314...

 done.


Found 123 bibliographic references in tmp_2509.25314/Main_XCavTNG-Comp.bbl.
Error retrieving bib data for Prunier2025a: 'author'
Retrieving document from  https://arxiv.org/e-print/2509.25317


...

T. Henning  ->  T. Henning  |  ['T. Henning']


Found 81 bibliographic references in tmp_2509.26430/toi375.bbl.
Error retrieving bib data for parsec: 'author'
Retrieving document from  https://arxiv.org/e-print/2509.26617
extracting tarball to tmp_2509.26617...

 done.


M. Benisty  ->  M. Benisty  |  ['M. Benisty']


Found 88 bibliographic references in tmp_2509.26617/sample631.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25314-b31b1b.svg)](https://arxiv.org/abs/2509.25314) | **X-ray shocks in the cool cores of galaxy clusters: insights from TNG-Cluster**  |
|| M. Prunier, F. Ubertosi, J. Hlavacek-Larrondo, <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *Accepted to MNRAS. The data of the TNG-Cluster simulation is now fully public and accessible at this URL this https URL*|
|**Abstract**|            Shock fronts driven by active galactic nuclei in galaxy cluster cores represent a promising mechanism to heat the intracluster gas by converting kinetic energy into thermal energy through gas compression, thereby offsetting radiative cooling. Despite their potential importance, such shocks are challenging to detect, requiring deep X-ray exposures, and have only been identified in ten clusters. We present the first systematic detection and characterization of AGN-driven shocks in simulated clusters from the TNG-Cluster magnetohydrodynamic cosmological zoom-in simulations of galaxies. TNG-Cluster exhibits a rich variety of X-ray structures, including realistic populations of X-ray cavities, as well as shocks, produced by its AGN feedback model, without collimated, relativistic jets, nor cosmic rays. We produce mock Chandra observations with deep exposure times, for a sample of 100 clusters, mass-matched (M$_{500c}=1.2$ - $8.5 \times 10^{14}$ M$_\odot$) to the ten observed clusters with shocks. Using observational techniques, we identify shocks through surface brightness edges fitted with broken power laws and associated density and temperature jumps. We detect 50 shocks in 30 of the 100 clusters, with ~35% hosting multiple shocks. These shocks lie within a hundred kiloparsec of the central SMBH, are weak (Mach number < 2, median ~ 1.1), and are associated with cavities in about half of the cases. Both in observations and in TNG-Cluster, shocks tend to be located at larger radii than cavities, with median offsets of 46 and 27 kpc, respectively. The observationally inferred shock powers are comparable to the cluster cooling luminosities (10$^{44-46}$ erg s$^{-1}$), suggesting that shocks in the simulation are crucial heating mechanisms. Our results indicate that shocks play a role as important as cavities in balancing cooling in cluster cores, acting isotropically and up to larger distances.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25338-b31b1b.svg)](https://arxiv.org/abs/2509.25338) | **Multi-modal atmospheric characterization of $β$ Pictoris b: Adding high-resolution continuum spectra from GRAVITY**  |
|| <mark>M. Ravet</mark>, et al. -- incl., <mark>G. Chauvin</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            We present the first VLTI/GRAVITY observations at R$_\lambda \sim 4000$ of $\beta$ Pic b. These four high S/N ($\sim$20) K-band spectra conserve both the pseudo-continuum and molecular absorption patterns. We analyze them with four self-consistent forward model grids (Exo-REM, ATMO, BT-Settl, Sonora) exploring $T_{\mathrm{eff}}$, log(g), metallicity, C/O, and $^{12}$CO/$^{13}$CO ratio. We also upgrade our forward modeling code \texttt{\textit{ForMoSA}} to account for the data multi-modality and combine the GRAVITY epochs with published 1-5 $\mu$m photometry, low- to medium-resolution spectra (0.9-7 $\mu$m), and high-resolution echelle spectra (2.1-5.2 $\mu$m). Sonora and Exo-REM are statistically preferred. Exo-REM yields $T_{\mathrm{eff}}$ $=1607.45^{+4.85}_{-6.20}$ K and log(g) $=4.46^{+0.02}_{-0.04}$ dex from GRAVITY alone, and $T_{\mathrm{eff}}$ $=1502.74^{+2.32}_{-2.14}$ K and log(g) $=4.00\pm0.01$ dex when including all datasets. Archival data significantly affect the retrieved parameters. C/O remains solar ($0.552^{+0.003}_{-0.002}$) while [M/H] reaches super-solar values (0.50$\pm$0.01). We report the first tentative constraint on log($^{12}$CO/$^{13}$CO) $\sim$1.12, though this remains inconclusive due to telluric residuals. Additionally, we estimate the luminosity to be log(L/L$_\odot$) $=-4.01^{+0.04}_{-0.05}$, implying a heavy-element content of up to $\sim$5% (20-80 M$_\oplus$) given the system age and dynamical mass measurements. Access to both continuum and molecular lines at K-band significantly impacts the metallicity, possibly owing to collision-induced absorption shaping the continuum. Echelle spectra do not dominate the final fit with respect to lower resolution data. Future multi-modal frameworks should include weighting schemes reflecting bandwidth and central wavelength coverage.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.26033-b31b1b.svg)](https://arxiv.org/abs/2509.26033) | **Edge-On Disk Study (EODS) III: Molecular Stratification in the Flying Saucer Disk**  |
|| A. Dutrey, et al. -- incl., <mark>D. Semenov</mark>, <mark>T. Henning</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| **|
|**Abstract**|            Context: Investigating the vertical distribution of molecular content in protoplanetary disks remains difficult in most disks mildly inclined along the line of sight. In contrast, edge-on disks provide a direct (tomographic) view of the 2D molecular brightness. Aims: We study the radial and vertical molecular distribution as well as the gas temperature and density by observing the Keplerian edge-on disk surrounding the Flying Saucer, a Class II object located in Ophiuchus. Methods: We use new and archival ALMA data to perform a tomography of $^{12}$CO, $^{13}$CO, C$^{18}$O, CN, HCN, CS, H$_2$CO, c-C$_3$H$_2$, N$_2$D$^+$, DCN and $^{13}$CS. We analyze molecular tomographies and model data using the radiative transfer code DiskFit. Results: We directly measure the altitude above the mid-plane for each observed species. For the first time, we unambiguously demonstrate the presence of a common molecular layer and measure its thickness: most molecules are located at the same altitude versus radius. Beyond CO, as predicted by chemical models, the CN emission traces the upper boundary of the molecular layer, whereas the deuterated species (DCN and N2D+) resides below one scale-height. Our best fits from DiskFit show that most observed transitions in the molecular layer are thermalized because their excitation temperature is the same, around 17-20 K. Conclusions: These long-integration observations clearly reveal a molecular layer predominantly located around 1-2 scale height, at a temperature above the CO freeze-out temperature. The deuterated molecules are closer to the mid-plane and N2D+ may be a good proxy for the CO snowline. Some molecules, such as CN and H2CO, are likely influenced by the disk environment, at least beyond the mm dust disk radius. The direct observation of the molecular stratification opens the door to detailed chemical modeling in this disk which appears representative of T Tauri disks.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.26430-b31b1b.svg)](https://arxiv.org/abs/2509.26430) | **A transiting hot Jupiter with two outer siblings orbiting an intermediate-mass post main-sequence star**  |
|| Y. Reinarz, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *resubmitted without line numbers*|
|**Abstract**|            Exoplanetary systems with multiple giant planets present an opportunity to understand planet formation, migration processes, and long-term system-wide dynamical interactions. In particular, they provide constraints to distinguish between smooth disk-driven migration or more dynamically excited system evolution pathways. We report the discovery and characterization of a unique multi-planet system hosting three gas giant planets orbiting the post-main sequence star TOI-375. The innermost planet, TOI-375 b, was initially detected by the TESS mission and then confirmed with photometric follow-up observations conducted using MEarth and LCOGT, and radial velocity measurements obtained with FEROS and CHIRON. The radial velocity data revealed the presence of two additional planetary candidates, TOI-375 c and TOI-375 d. We find that TOI-375 b is a hot Jupiter with an orbital period of $9.45469 \pm 0.00002$ days, mass $0.745 \pm 0.053,M_\mathrm{J}$, radius $0.961 \pm 0.043, R_\mathrm{J}$, and eccentricity $0.087 \pm 0.042$. The outer two planets, TOI-375 c and TOI-375 d, are warm-cold and cold Jupiters with orbital periods of $115.5^{+2.0}_{-1.6}$ days and $297.9^{+28.9}_{-18.6}$ days, and minimum masses of $2.11 \pm 0.22, M_\mathrm{J}$ and $1.40 \pm 0.28, M_\mathrm{J}$, respectively. In terms of formation and overall system architecture, the physical properties of TOI-375 b are consistent with the core accretion scenario, while the current configuration of the system could be explained by both disk-driven and high-eccentricity migration scenarios. The discovery of TOI-375 as the first known system hosting three or more fully evolved gas giants, with at least one transiting planet, makes it an excellent candidate for testing formation and migration theories.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25335-b31b1b.svg)](https://arxiv.org/abs/2509.25335) | **The SAGA Survey. VI. The Size-Mass Relation for Low-Mass Galaxies Across Environments**  |
|| Y. Asali, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *30 pages, 14 figures, 2 tables*|
|**Abstract**|            We investigate how Milky Way-like environments influence the sizes and structural properties of low-mass galaxies by comparing satellites of Milky Way analogs from the Satellites Around Galactic Analogs (SAGA) Survey with two control samples: an environmentally agnostic population from the SAGA background (SAGAbg) sample and isolated galaxies from the SDSS NASA-Sloan Atlas. All sizes and structural parameters are measured uniformly using pysersic to ensure consistency across samples. We find the half-light sizes of SAGA satellites are systematically larger than those of isolated galaxies, with the magnitude of the offset ranging from 0.05 to 0.12 dex (10-24%) depending on the comparison sample and completeness cuts. This corresponds to physical size differences between 85-200 pc at 10^7.5 solar masses and 220-960 pc at 10^10 solar masses. This offset persists among star-forming galaxies, suggesting that environment can influence the structure of low-mass galaxies even before it impacts quenching. The intrinsic scatter in the size-mass relation is lower for SAGA satellites than isolated galaxies, and the Sérsic index distributions of satellites and isolated galaxies are similar. In comparison to star-forming satellites, quenched SAGA satellites have a slightly shallower size-mass relation and rounder morphologies at low-mass, suggesting that quenching is accompanied by structural transformation and that the processes responsible differ between low- and high-mass satellites. Our results show that environmental processes can imprint measurable structural differences on satellites in Milky Way-mass halos.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25362-b31b1b.svg)](https://arxiv.org/abs/2509.25362) | **Machine Learning the Dark Matter Halo Mass of the Milky Way**  |
|| E. Hayati, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| **|
|**Abstract**|            Although numerous dynamical techniques have been developed to estimate the total dark matter halo mass of the Milky Way, it remains poorly constrained, with typical systematic uncertainties of 0.3 dex. In this study, we apply a neural network-based approach that achieves high mass precision without several limitations that have affected past approaches; for example, we do not assume dynamical equilibrium, nor do we assume that neighboring galaxies are bound satellites. Additionally, this method works for a broad mass range, including for halos that differ significantly from the Milky Way. Our model relies solely on observable dynamical quantities, such as satellite orbits, distances to larger nearby halos, and the maximum circular velocity of the most massive satellite. In this paper, we measure the halo mass of the Milky Way to be $\log_{10} M_\mathrm{vir}/\Msun = 12.20^{+0.163}_{-0.138}$. Future studies in this series will extend this methodology to estimate the dark matter halo mass of M31, and develop new neural networks to infer additional halo properties including concentration, assembly history, and spin axis.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25610-b31b1b.svg)](https://arxiv.org/abs/2509.25610) | **Nonuniform Water Distribution in Jupiter's Mid Latitudes: Influence of Precipitation and Planetary Rotation**  |
|| H. Ge, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *Published in PNAS on Sep. 29, 2025 (this http URL). Movies can be found in this https URL. SI Appendix can be found on PNAS website*|
|**Abstract**|            Knowing the composition of Jupiter's atmosphere is crucial for constraining Jupiter's bulk metallicity and formation history. Yet, constraining Jupiter's atmospheric water abundance is challenging due to its potential non-uniform distribution. Here, we explicitly resolve the water hydrological cycle in Jupiter's mid-latitudes using high-resolution simulations. Falling precipitation leads to a significant large-scale depletion of water vapor beneath the lifting condensation level. A non-uniform water vapor distribution emerges in the mid-latitude simulation with a changing Coriolis parameter across latitudes and spatially uniform cooling and heating. Water abundance at the 7-bar level varies by up to a factor of ten across latitudes, from sub-solar to super-solar values. We propose that nonlinear large-scale eddies and waves tend to drift air parcels across latitudes along constant potential vorticity (PV) surfaces, thereby sustaining latitudinal dependencies in water vapor and the interplay between water distribution and large-scale dynamics. Therefore, water distribution is influenced by the vertical structure of density stratification and changing Coriolis parameter across Jupiter's mid-latitudes, as quantified by PV. Additionally, the water hydrological cycle amplifies the specific energy of air parcels through the latent heat effect, thereby slowing down vertical mixing with a latent heat flux. The horizontal gradient of water is expected to be more pronounced with a super-solar water abundance. We suggest that similar interplays between precipitating condensates, planetary rotation, and distribution of condensable species generally exist in the weather layer of fast-rotating giant planets. The ongoing Juno mission and future Uranus mission may further reveal the non-uniform distribution of condensed species and their interplay with large-scale dynamics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.25726-b31b1b.svg)](https://arxiv.org/abs/2509.25726) | **The Simons Observatory: Studies of Phase Drift in RF Transmission Lines from the First Large-Scale Deployment of Microwave Frequency Multiplexing for Cosmology**  |
|| T. P. Satterthwaite, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *6 pages, 7 figures. Proceedings of the 21st International Conference on Low Temperature Detectors (LTD 2025)*|
|**Abstract**|            Fulfilling the science goals of the Simons Observatory, a state-of-the-art cosmic microwave background (CMB) experiment, has required deploying tens of thousands of superconducting bolometers. Reading out data from the observatory's more than 67,000 transition-edge sensor (TES) detectors while maintaining cryogenic conditions requires an effective multiplexing scheme. The SLAC microresonator radio frequency (SMuRF) electronics have been developed to provide the warm electronics for a high-density microwave frequency multiplexing readout system, and this system has been shown to achieve multiplexing factors on the order of 1,000. SMuRF has recently been deployed to the Simons Observatory, which is located at 5,200 m on Cerro Toco in Chile's Atacama Desert. As the SMuRF system is exposed to the desert's diurnal temperature swings, resulting phase drift in RF transmission lines may introduce a systematic signal contamination. We present studies of phase drift in the room-temperature RF lines of the Simons Observatory's 6 m large-aperture telescope, which hosts the largest deployment to date of TES microwave frequency multiplexing to a single telescope. We show that these phase drifts occur on time scales which are significantly longer than sky scanning, and that their contribution to on-sky in-transition detector noise is within the readout noise budget.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.26283-b31b1b.svg)](https://arxiv.org/abs/2509.26283) | **On the Ultra-Long Gamma-Ray Transient GRB 250702B/EP250702**  |
|| J.-P. Zhang, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-01*|
|*Comments*| *16 pages, 6 figures. Originally submitted to ApJL on September 2, 2025. Comments welcome!*|
|**Abstract**|            GRB 250702B/EP250702a is an interesting long-duration gamma-ray transient whose nature is in debate. To obtain a full picture in gamma-ray band, we implement a comprehensive targeted search of burst emission in a wide window of 30 days jointly with Insight-HXMT, GECAM and Fermi/GBM data within the ETJASMIN framework. In gamma-ray band, we find there is a 50-second precursor about 25 hours before the 4-hour main burst, which generally consists of 4 emission episodes. Remarkably, we find that the soft X-ray emission (after the main burst) decays as a power-law with start time aligning with the last episode of main emission and index of -5/3 perfectly consistent with the canonical prediction of fallback accretion. We conclude that the properties of precursor, main burst and the following soft X-ray emission strongly support the atypical collapsar Ultra-Long Gamma-Ray Burst (ULGRB) scenario rather than the Tidal Disruption Event (TDE), and all these gamma-ray and soft X-ray emission probably originate from relativistic jet whose luminosity is dominated by the fallback accretion rate during the death collapse of a supergiant star.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.25314/./figure_panel_shock/Panel_Shock_1.png', 'tmp_2509.25314/./figure_panel_shock/Panel_Shock_2.png', 'tmp_2509.25314/./figures/Demographics.png', 'tmp_2509.25314/./figures/Demographics_pie.png', 'tmp_2509.25314/./figures/Power_Lcool_Cum_Each.png', 'tmp_2509.25314/./figures/Power_Lcool_Cum_Total.png']
copying  tmp_2509.25314/./figure_panel_shock/Panel_Shock_1.png to _build/html/
copying  tmp_2509.25314/./figure_panel_shock/Panel_Shock_2.png to _build/html/
copying  tmp_2509.25314/./figures/Demographics.png to _build/html/
copying  tmp_2509.25314/./figures/Demographics_pie.png to _build/html/
copying  tmp_2509.25314/./figures/Power_Lcool_Cum_Each.png to _build/html/
copying  tmp_2509.25314/./figures/Power_Lcool_Cum_Total.png to _build/html/
exported in  _build/html/2509.25314.md
    + _build/html/tmp_2509.25314/./figure_panel_shock/Panel_Shock_1.png
    + _build/html/tmp_2509.25314/./figure_panel_shock/Panel_Shock_2.png
    + _build/html/tmp_2509.25314/./figur

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\msun}{ M_{\odot}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand\francesco{#1}$
$\newcommand\julie{#1}$
$\newcommand\annalisa{#1}$</div>



<div id="title">

# ${X-ray shocks in the cool cores of galaxy clusters: insights from TNG-Cluster}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.25314-b31b1b.svg)](https://arxiv.org/abs/2509.25314)<mark>Appeared on: 2025-10-01</mark> -  _Accepted to MNRAS. The data of the TNG-Cluster simulation is now fully public and accessible at this URL this https URL_

</div>
<div id="authors">

M. Prunier, F. Ubertosi, J. Hlavacek-Larrondo, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** Shock fronts driven by active galactic nuclei in galaxy cluster cores represent a promising mechanism to heat the intracluster gas by converting kinetic energy into thermal energy through gas compression, thereby offsetting radiative cooling. Despite their potential importance, such shocks are challenging to detect, requiring deep X-ray exposures, and have only been identified in ${ten massive clusters}$ . We present the first systematic detection and characterization of AGN-driven shocks in simulated clusters from the TNG-Cluster magnetohydrodynamic cosmological zoom-in simulations of galaxies. TNG-Cluster exhibits a rich variety of spatially-resolved X-ray structures, including realistic populations of X-ray cavities, as well as shocks, ${produced by its AGN feedback model, without collimated, relativistic jets, nor cosmic rays.}$ We produce mock _Chandra_ observations with deep, 600-ks exposure times, for a selected sample of 100 clusters, mass-matched (M $_\text{500c}=1.2$ - $8.5 \times 10^{14}$ M $_\odot$ ) to the ten observed clusters with shocks. Using observational techniques, we identify shocks through surface brightness edges fitted with broken power laws and associated density and temperature jumps. We detect 50 shocks in 30 of the 100 clusters, with $\sim$ 35 per cent hosting multiple shocks. These shocks typically lie within a hundred kiloparsec of the central SMBH, are weak (Mach number $<$ 2, median $\sim$ 1.1), and ${are associated with X-ray cavities in about half of the cases.}$ Both in observations and in TNG-Cluster, shocks tend to be located at larger radii than cavities, with median offsets of 46 kpc and 27 kpc, respectively. The observationally inferred shock powers are comparable to the cluster cooling luminosities (10 $^{44-46}$ erg s $^{-1}$ ), suggesting that shocks in the simulation are a crucial heating mechanism. Our results ${indicate}$ that shocks play a role as important as cavities in balancing cooling in cluster cores, acting isotropically and up to larger distances.

</div>

<div id="div_fig1">

<img src="tmp_2509.25314/./figure_panel_shock/Panel_Shock_1.png" alt="Fig6.1" width="50%"/><img src="tmp_2509.25314/./figure_panel_shock/Panel_Shock_2.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Gallery of {simulated galaxy clusters from TNG-Cluster} exhibiting X-ray shocks. Mock _Chandra_ surface brightness images of the central $200\times200$ kpc region of six selected clusters in TNG-Cluster, alongside images processed with unsharp masking or Gaussian gradient magnitude filters to highlight the shock fronts. Images are centered on the AGN position. White dashed arrows mark identified shock fronts, while the green arrow points the specific shock for which we show, in Fig. \ref{fig:panel_shocks_fit_temp_jump}, the surface brightness profile and associated temperature jump. The gallery illustrates the diversity of AGN-driven shocks in TNG-Cluster cores, including features located at cavity edges or at larger radii, and exhibiting a range of morphologies, from arc-like fronts with varying opening angles to complete ellipses. (*fig:panel_shocks*)

</div>
<div id="div_fig2">

<img src="tmp_2509.25314/./figures/Demographics.png" alt="Fig8.1" width="50%"/><img src="tmp_2509.25314/./figures/Demographics_pie.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Demographics of clusters with and without identified shocks and X-ray cavities in the TNG-Cluster subsample of 100 clusters (at snapshot 99). _Left:_ Distribution of the number of shocks (mint green) or X-ray cavities (gray) per cluster, showing the percentage of clusters with none, one, two, three, or four detected features. _Right:_ Among the 45 clusters with at least one identified shock or cavity, the fraction hosting only shocks, only cavities, or both. Shocks are detected in 30 out of the 100 clusters in our subsample, indicating that they are {reasonably} common features in simulated clusters. While shocks frequently appear in clusters that also host X-ray cavities, about 18\% of the detected shocks are found in clusters where no cavities were previously identified in Prunier2025a. (*fig:panel*)

</div>
<div id="div_fig3">

<img src="tmp_2509.25314/./figures/Power_Lcool_Cum_Each.png" alt="Fig10.1" width="50%"/><img src="tmp_2509.25314/./figures/Power_Lcool_Cum_Total.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** _Left:_ Total shock (P$_\text{shock,tot}$) and X-ray cavity power (P$_\text{cav,tot}$) vs. cooling luminosity (L$_\text{cool}$, for a cooling radius of 7.7 Gyr) for the 45 clusters with either identified shock or X-ray cavities (or both) in the TNG-Cluster sample (mint green circle, respectively, blue square markers) and the observational sample (values from the literature see Table \ref{tab:shocks}). Each point represents the sum of shocks or X-ray cavities in the considered system. The dashed gray diagonal marks the 1:1 correlation between power and cooling luminosity. The side histograms display the distribution of each quantity, with the median. _Right:_ Total power from the summed shock(s) and X-ray cavity(ies) vs. cooling luminosity for each of the 22 clusters hosting both, and for the observational sample (triangle). In the TNG-Cluster sample, the distributions of shock and cavity power are similar and generally comparable to the cooling luminosity. For clusters with both features, the total power typically exceeds L$_\text{cool}$ by a factor of 1 to 10, which is lower than in most observed systems (10 to 100 $\times$ L$_\text{cool}$).  (*fig:pcav_pshock_lcool*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.25314"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Rjup}{\ensuremath{\mathrm{R_{Jup}}}\xspace}$
$\newcommand{\MJup}{\ensuremath{\mathrm{M_{Jup}}}\xspace}$
$\newcommand{\Teff}{\ensuremath{\mathrm{T_{eff}}}\xspace}$
$\newcommand{\Mjup}{\MJup}$
$\newcommand{\Msun}{\ensuremath{\mathrm{M_{\odot}}}}$
$\newcommand{\kms}{km/s\xspace}$
$\newcommand{\um}{\upmum\xspace}$
$\newcommand{\Mdot}{\ensuremath{\dot{\textrm{M}}}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Multi-modal atmospheric characterization of $\beta$ Pictoris b: Adding high-resolution continuum spectra from GRAVITY

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.25338-b31b1b.svg)](https://arxiv.org/abs/2509.25338)<mark>Appeared on: 2025-10-01</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

<mark>M. Ravet</mark>, et al. -- incl., <mark>G. Chauvin</mark>

</div>
<div id="abstract">

**Abstract:** The characterization of giant exoplanets such as $\beta$ Pictoris b (hereafter $\beta$ Pic b)  is now routinely performed with multiple spectrographs and imagers  exploring different spectral bandwidths and resolutions, allowing for atmospheric retrieval of spectra with or without the conservation of the planet spectral continuum. The accounting of data multi-modality in the analysis can provide a more comprehensive determination of the planets physical and chemical properties and inform on their formation history. We present the first VLTI observations at R $_{\lambda}\sim$ 4 000 of $\beta$ Pic b, the first obtained on an exoplanet with GRAVITY at such resolution. We upgrade the forward modelling code \texttt{_ForMoSA_} to account for the data multi-modality, including low- medium- and high-resolution spectroscopy both based on direct model-data comparison and the analysis of cross-correlation signals. We use \texttt{_ForMoSA_} to refine the constraints on the atmospheric properties of the exoplanet and evaluate the sensitivity on the retrieved values to the input dataset. We obtain four high signal-to-noise (S/N $\sim$ 20) spectra of $\beta$ Pic b at K-band with GRAVITY at R $_{\lambda}\sim$ 4 000 conserving both the pseudo-continuum and the pattern of molecular absorptions. We use \texttt{_ForMoSA_} with four grids of self-consistent forward models (Exo-REM, ATMO, BT-Settl, and Sonora) exploring different $\Teff$ , log(g), metallicity,  C/O, and $^{12}$ CO/ $^{13}$ CO ratio. We then combine the GRAVITY spectra with published 1-5 µm photometry (NaCo, VisAO, NICI, SPHERE), low- to medium- resolution ( $R_{\lambda} \leq 700$ broad band, 0.9-7 µm) spectra, and echelle spectra covering narrower bandwidths ( $R_{\lambda}\sim$ 100 000, 2.1-5.2 µm). Sonora and Exo-REM are statistically preferred among all four models, regardless of the dataset used. Exo-REM predicts $\Teff$  $=1607.45^{+4.85}_{-6.20}$ K and log(g) $=4.46^{+0.02}_{-0.04}$ dex when using only the GRAVITY epochs and $\Teff$  $=1502.74^{+2.32}_{-2.14}$ K and log(g) $=4.00\pm0.01$ dex when incorporating all available datasets. The inclusion of archival data significantly affects all retrieved posteriors. Using all datasets, C/O mostly remains solar (C/O = $0.552^{+0.003}_{-0.002}$ ) while [ M/H ] reaches super-solar values (0.50 $\pm$ 0.01). We report the first tentative constrain on the isotopic ratio log( $^{12}$ CO/ $^{13}$ CO) = 1.12 $^{+0.11}_{-0.08}$ in $\beta$ Pic b's atmosphere; however, this detection remains inconclusive due to telluric residuals affecting both the GRAVITY and SINFONI data. Additionally, we estimate the bolometric luminosity as log(L/L $\mathrm{_{\odot}}$ ) = -4.01 $^{+0.04}_{-0.05}$ dex. Using a system age of 23 $\pm$ 3 Myr, this bolometric luminosity and the constraints on the dynamical mass of $\beta$ Pic b we constrain the maximum of heavy-element content of the planet to be on the order of 5 \% (20-80 $\ensuremath{\mathrm{M_{Earth}}}$ $\xspace$ ). The joint access to the pseudo-continuum and molecular lines at K-band as provided by GRAVITY have a significant impact on the retrieved metallicity, possibly owing to the collision-induced absorption driving the continuum shape of K-band. Echelle spectra do not dominate the final fit with respect to lower resolution data covering a broader portion of the spectral energy distribution and the latter keeps encapsulating more robust information on $\Teff$ . Future multi-modal framework should include a weighting scheme accounting for the bandwidth and central wavelength of the observations.

</div>

<div id="div_fig1">

<img src="tmp_2509.25338/./IMAGES/rv_evo.png" alt="Fig5" width="100%"/>

**Figure 5. -** Orbital predictions from lacour_mass_2021 (dotted line) vs. retrieved radial velocities (colored bars) for each epoch,  using forward modeling (here BT-Settl). The four GRAVITY epochs were fitted in parallel using MOSAIC,  and we can see that the observation on the 2019/11/09 is significantly shifted. (*all_rvs*)

</div>
<div id="div_fig2">

<img src="tmp_2509.25338/./IMAGES/ALL_spec_CRIRES.png" alt="Fig15" width="100%"/>

**Figure 15. -** Results of the forward modeling of $\beta$ Pic b using the most recent spectro/photometric observations of the planet (including CRIRES$_+$) using BT-Settl. _Top panel:_ spectra (points) and photometry (squares) of $\beta$ Pic b alongside a R$_{\lambda\sim$ 4 000 spectrum extracted using the best fit parameters for the inversion. Observations have been re-normalized using the analytic scaling factors computed during the inversion. Associated residuals are shown below. Dotted lines represents the $\pm\sigma$ (68$\%$) confidence interval. _Bottom sub-panels:_ zoom versions of the top panel on GRAVITY (upper left), SINFONI (upper right), CRIRES$_+$-K (lower left) and CRIRES$_+$-M (lower right). Similarly to landman__2024, each CRIRES$_+$ spectra are shown smoothed by their retrieved broadening to show the planet features more clearly.} (*ALL_MOSAIC_CRIRES_spec*)

</div>
<div id="div_fig3">

<img src="tmp_2509.25338/./IMAGES/PT_clouds_merged.png" alt="Fig3" width="100%"/>

**Figure 3. -** Retrieved best-fit pressure-temperature profiles when using GRAVITY (light blue) and all datasets except CRIRES$_+$ (dark blue) for Exo-REM forward modeling. Shaded regions represent the 1 and 2 $\sigma$ confidence interval, respectively. They are compared with the cloud opacities of $Mg_2SiO_4$ (dashed brown) and Fe (dashed gray) defined as $d\tau=\sigma_c n dz$ with $\sigma_c$ the effective extinction cross-section of the cloud (in m$^2$), $n$ its concentration (in particles/m$^3$) and $dz$ the thickness of the atmospheric layer (in m). The red curve represents the photosphere (region from which most thermal emission originates). In practice, we used a similar approach as charnay_self-consistent_2018 and constrained the pressure-temperature between the minimum and maximum of the brightness temperature computed between 0.625 to 10 µm. (*Free_MOSAIC_PT*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.25338"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vlsr}{v_{\rm lsr}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\mdot}{\dot{M}}$
$\newcommand{\Msun}{ \mathrm{M}_\odot}$
$\newcommand{\Msunpyr}{\unit{M_\odot yr^{-1}}}$
$\newcommand{\coto}{\mbox{CO(2--1)}}$
$\newcommand{\cott}{\mbox{CO(3--2)}}$
$\newcommand{\tcott}{\mbox{^{13}CO(2--1)}}$
$\newcommand{\csff}{\mbox{CS(5--4)}}$
$\newcommand{\csss}{\mbox{CS(7--6)}}$
$\newcommand{\cnto}{\mbox{CN(2--1)}}$
$\newcommand{\cntt}{\mbox{CN(3--2)}}$
$\newcommand{\textsg}[1]{\textcolor{red}{SGu: #1}}$
$\newcommand{\textad}[1]{\textcolor{magenta}{ADu: #1}}$
$\newcommand{\textds}[1]{\textcolor{blue}{DS: #1}}$</div>



<div id="title">

# Edge-On Disk Study (EODS) III: Molecular Stratification in the Flying Saucer Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.26033-b31b1b.svg)](https://arxiv.org/abs/2509.26033)<mark>Appeared on: 2025-10-01</mark> - 

</div>
<div id="authors">

A. Dutrey, et al. -- incl., <mark>D. Semenov</mark>, <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Investigating the vertical distribution of molecular content in protoplanetary disks remains difficult in most disks mildly inclined along the line of sight.In contrast, edge-on disks provide a direct (tomographic) view of the 2D molecular brightness. We study the radial and vertical molecular distribution as well as the gas temperature and density by observing the Keplerian edge-on disk surrounding the Flying Saucer, a Class II object located in Ophiuchus. We use new and archival ALMA data to perform a tomography of $^{12}$ CO, $^{13}$ CO, C $^{18}$ O, CN, HCN, CS, $H_2$ CO, c-$C_3$ $H_2$ , $N_2$ D $^+$ , DCN and $^{13}$ CS. We analyze molecular tomographies and model data using the radiative transfer code DiskFit . We directly measure the altitude above the mid-plane for each observed species. For the first time, we unambiguously demonstrate the presence of a common molecular layer and measure its thickness: most molecules are located at the same altitude versus radius. Beyond CO, as predicted by chemical models, the CN emission traces the upper boundary of the molecular layer, whereas the deuterated species (DCN and $N_2$ D $^+$ ) resides below one scale-height. Our best fits from DiskFit show that most observed transitions in the molecular layer are thermalized  because their excitation temperature is the same, around $\sim$ 17-20 K. These long-integration observations clearly reveal a molecular layer predominantly located around 1-2 scale height, at a temperature above the CO freeze-out temperature. The deuterated molecules are closer to the mid-plane and $N_2$ D $^+$ may be a good proxy for the CO snowline. Some molecules, such as CN and $H_2$ CO, are likely influenced by the disk environment, at least beyond the mm dust disk radius. The direct observation of the molecular stratification opens the door to detailed chemical modeling in this disk which appears representative of T Tauri disks.

</div>

<div id="div_fig1">

<img src="tmp_2509.26033/./altitude.png" alt="Fig5.1" width="50%"/><img src="tmp_2509.26033/./true-width.png" alt="Fig5.2" width="50%"/>

**Figure 5. -** Top: Altitude $A(r)$ of the molecular layer as a function of radius. The black curve is the HCN(4-3) altitude for comparison. The cyan region indicates the
the approximate size of the dust disk. The dotted line corresponds to z/r = 0.2.
Bottom: Deconvolved thickness of the molecular layer. The deconvolution is done assuming Gaussian shapes, using the Clean beam minor axis since synthesized beams are elongated almost parallel to the disk plane. The black curve is the HCN(4-3) thickness for comparison. The cyan bar marks the edge of the dust disk. (*fig:height-tomo*)

</div>
<div id="div_fig2">

<img src="tmp_2509.26033/./lines-over-13co.png" alt="Fig1" width="100%"/>

**Figure 1. -** $^{13}$CO tomography in false color with superimposition of the tomographies, in contours, of the HCN 4-3, CS 7-6, CN at 340.348 GHz, p-$H_2$CO at 218.222 GHz, DCN 3-2 and $N_2$D$^+$ 3-2. Contours are in steps
of 1.5 K for HCN, CS, CN and p-$H_2$CO, 0.75 K for DCN, and 0.375 K for $N_2$D$^+$. The grey ellipses indicate the impact of line width on the effective beam at 80 and 240 au (*fig:tomo-super*)

</div>
<div id="div_fig3">

<img src="tmp_2509.26033/./profile100.png" alt="Fig2" width="100%"/>

**Figure 2. -** Profiles resulting from DiskFit at 100 au.
The green curve is the temperature profile, and the cyan curves indicate the $H_2$ density structure (thin: under isothermal assumption at the mid-plane temperature (10 K), thick: using the full temperature profile) that were derived from the CO analysis by [Guilloteau, Denis-Alpizar and Dutrey (2025)]().
The other colored curves are the molecular density profiles for CS (red), HCN (black), CN 2-1 (blue) and 3-2 (magenta), using parameters from Table \ref{tab:models}. The corresponding horizontal bars indicate the measured temperatures and the density weighted
 average height of the emission. (*fig:profiles*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.26033"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\feh}{\ensuremath{{\rm[Fe/H]}}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\zaspe}{\textsf{ZASPE}}$
$\newcommand{\ceres}{\textsf{CERES}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}}$
$\newcommand{\kms}{\ensuremath{{\rm km s^{-1}}}}$
$\newcommand{\mjup}{\ensuremath{{\rm M_{J}}}}$
$\newcommand{\mearth}{\ensuremath{{\rm M}_{\oplus}}}$
$\newcommand{\mpl}{\ensuremath{{\rm M_p}}}$
$\newcommand{\rjup}{\ensuremath{{\rm R_J}}}$
$\newcommand{\rpl}{\ensuremath{{\rm R_P}}}$
$\newcommand{\rstar}{\ensuremath{{\rm R}_{\star}}}$
$\newcommand{\mstar}{\ensuremath{{\rm M}_{\star}}}$
$\newcommand{\lstar}{\ensuremath{{\rm L}_{\star}}}$
$\newcommand{\rsun}{\ensuremath{{\rm R}_{\odot}}}$
$\newcommand{\msun}{\ensuremath{{\rm M}_{\odot}}}$
$\newcommand{\lsun}{\ensuremath{{\rm L}_{\odot}}}$
$\newcommand{\stlum}{\ensuremath{2.33 \pm 0.11 }}$
$\newcommand{\stage}{\ensuremath{2.92_{-0.73}^{+0.80} }}$
$\newcommand{\av}{\ensuremath{0.226_{-0.058}^{+0.056} }}$
$\newcommand{\stteff}{\ensuremath{6295 \pm 77 }}$
$\newcommand{\stfeh}{\ensuremath{0.00 \pm 0.05 }}$
$\newcommand{\stlogg}{\ensuremath{4.291 \pm 0.025 }}$
$\newcommand{\stvsini}{\ensuremath{7.80 \pm 0.19 }}$
$\newcommand{\stmstan}{\ensuremath{1.170\pm0.06}}$
$\newcommand{\strstan}{\ensuremath{1.282\pm0.03}}$
$\newcommand{\per}{\ensuremath{11.23660 \pm 0.00011}}$
$\newcommand{\bb}{\ensuremath{0.723_{-0.024}^{+0.018}}}$
$\newcommand{\rr}{\ensuremath{0.0942_{-0.0012}^{+0.0010}}}$
$\newcommand{\omegapost}{\ensuremath{1.230_{-0.063}^{+0.063}}}$
$\newcommand{\ecc}{\ensuremath{0.435\pm 0.024 }}$
$\newcommand{\logm}{\ensuremath{5.973_{-0.056}^{+0.054}}}$
$\newcommand{\mptess}{\ensuremath{1.236_{-0.067}^{+0.069}}}$
$\newcommand{\rptess}{\ensuremath{0.1202 \pm 0.003 }}$
$\newcommand{\rptessrjup}{\ensuremath{1.170\pm 0.03 }}$
$\newcommand{\stm}{\ensuremath{1.181\pm 0.058 }}$
$\newcommand{\str}{\ensuremath{1.28_{-0.03}^{+0.03}}}$
$\newcommand{\aone}{\ensuremath{1.58\pm 0.19 }}$
$\newcommand{\azero}{\ensuremath{-6.4_{-8.2}^{+8.1}}}$
$\newcommand{\bone}{\ensuremath{0.0204_{-0.0040}^{+0.0039}}}$
$\newcommand{\bzero}{\ensuremath{-0.00197_{-0.00032}^{+0.00033}}}$
$\newcommand{\etaferos}{\ensuremath{13.5_{-3.1}^{+3.3}}}$
$\newcommand{\etacoralie}{\ensuremath{45.1_{-11.4}^{+12.4}}}$
$\newcommand{\etachiron}{\ensuremath{21.7_{-7.6}^{+8.0}}}$
$\newcommand{\etassmko}{\ensuremath{0.00114_{-0.00010}^{+0.00011}}}$
$\newcommand{\gammaferos}{\ensuremath{-42.8_{-8.6}^{+8.8}}}$
$\newcommand{\gammacoralie}{\ensuremath{-26.9_{-15.4}^{+14.7}}}$
$\newcommand{\gammachiron}{\ensuremath{-20.9_{-10.6}^{+10.7}}}$
$\newcommand{\gammanres}{\ensuremath{-28.5_{-11.4}^{+11.4}}}$
$\newcommand{\uone}{\ensuremath{0.50_{-0.27}^{+0.19}}}$
$\newcommand{\utwo}{\ensuremath{-0.06_{-0.23}^{+0.33}}}$
$\newcommand{\rone}{\ensuremath{0.0942_{-0.0012}^{+0.0010}}}$
$\newcommand{\rtwo}{\ensuremath{0.723_{-0.024}^{+0.018}}}$
$\newcommand{\strho}{\ensuremath{0.80_{-0.06}^{+0.06}}}$
$\newcommand{\tzero}{\ensuremath{2458547.47448_{-0.00029}^{+0.00028}}}$
$\newcommand{\sma}{\ensuremath{0.1038_{-0.0017}^{+0.0017}}}$
$\newcommand{\inc}{\ensuremath{87.63_{-0.1}^{+0.11}}}$
$\newcommand{\teqv}{\ensuremath{1252\pm 21 }}$
$\newcommand{\stnameTIC}{TIC~280206394}$
$\newcommand{\rhopl}{\ensuremath{{\rm \rho_P}}}$
$\newcommand{\rhostar}{\ensuremath{\rho_*}}$
$\newcommand{\gccm}{\ensuremath{\mathrm{g} \mathrm{cm}^{-3}}}$</div>



<div id="title">

# A transiting hot Jupiter with two outer siblings orbiting an intermediate-mass post main-sequence star

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.26430-b31b1b.svg)](https://arxiv.org/abs/2509.26430)<mark>Appeared on: 2025-10-01</mark> -  _resubmitted without line numbers_

</div>
<div id="authors">

Y. Reinarz, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Exoplanetary systems with multiple giant planets present an opportunity to understand planet formation, migration processes, and long-term system-wide dynamical interactions. In particular, they provide constraints to distinguish between smooth disk-driven migration or more dynamically excited system evolution pathways.We report the discovery and characterization of a unique multi-planet system hosting three gas giant planets, orbiting the post-main sequence star TOI-375. The innermost planet, TOI-375 b, was initially detected by the _TESS_ mission, and then confirmed with photometric follow-up observations conducted using _MEarth_ and _LCOGT_ , and radial velocity measurements obtained with _FEROS_ and _CHIRON_ . The radial velocity data revealed the presence of two additional planetary candidates, TOI-375 c and TOI-375 d. We find that TOI-375 b is a hot Jupiter with an orbital period of $\ensuremath{9.45469 \pm 0.00002}$ days, mass $\ensuremath{0.745 \pm 0.053 M_\mathrm{J}}$ , radius $\ensuremath{0.961 \pm 0.043 R_\mathrm{J}}$ , and eccentricity $\ensuremath{0.087 \pm 0.042}$ . The outer two planets, TOI-375 c and TOI-375 d, are warm-cold and cold Jupiters with orbital periods of $\ensuremath{115.5^{+2.0}_{-1.6}}$ days and $\ensuremath{297.9^{+28.9}_{-18.6}}$ days, and minimum masses of $\ensuremath{2.11 \pm 0.22 M_\mathrm{J}}$ and $\ensuremath{1.40 \pm 0.28 M_\mathrm{J}}$ , respectively.In terms of formation and overall system architecture, the physical properties of TOI-375 b are consistent with the core accretion scenario, while the current configuration of the system could be explained by both disk-driven and high-eccentricity migration scenarios. The discovery of TOI-375 as the first known system hosting three or more fully evolved gas giants, with at least one transiting planet, makes it an excellent candidate for testing formation and migration theories.

</div>

<div id="div_fig1">

<img src="tmp_2509.26430/./figures/HR_diagram_GHSs.png" alt="Fig1" width="100%"/>

**Figure 1. -** Position of TOI-375 in the HR diagram (black star), using the stellar parameters derived here (see Table 3). For comparison, the position of known giant stars hosting transiting giant planets are overplotted. Two different \textsf{PARSEC} models  (parsec)  for 1.0 and 1.45 $\msun$, are overplotted. The solid and dotted lines correspond to Z$_\star$=0.017 and 0.020, respectively. (*fig:HR_diagram*)

</div>
<div id="div_fig2">

<img src="tmp_2509.26430/./figures/lcs_trim.png" alt="Fig4" width="100%"/>

**Figure 4. -** In green, we show the Phase-folded light curves from each instrument (or sector in the case of _TESS_) with the best-fit transit model for TOI-375 b derived using \textsf{juliet}. In blue, we show the potential single transit event of one of the outer planets on _TESS_ sector 2. Data points are shown in their original time sampling (small green or blue points) and binned in phase with 6-minute intervals (white markers). The solid black line represents the best-fit transit model. We do not include the transit of TOI-375 c in the final model as there is not enough evidence to support it is from true planetary origin. (*fig:transit_models*)

</div>
<div id="div_fig3">

<img src="tmp_2509.26430/./figures/soar.png" alt="Fig2" width="100%"/>

**Figure 2. -** Speckle autocorrelation function for TOI-375, obtained with the _SOAR_ telescope. The black dots represent the $5\sigma$ contrast curve, and the solid line shows a linear fit to the data at separations smaller and larger than $\sim 0.2$ arcseconds. (*fig:soar*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.26430"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\todo}[1]{\textcolor{red}{#1}}$
$\newcommand{\Michael}[1]{\textcolor{violet}{#1}}$
$\newcommand{\final}[1]{\textcolor{green}{#1}}$
$\newcommand{\irchi}[2]{\raisebox{\depth}{#1\chi}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\MJ}{M_\mathrm{J}}$
$\newcommand{\Mp}{M_\mathrm{p}}$
$\newcommand{\Macc}{\dot{M}_\mathrm{acc}}$
$\newcommand{\RJ}{R_\mathrm{J}}$
$\newcommand{\Rp}{R_\mathrm{p}}$
$\newcommand{\Teff}{T_{\text{eff}}}$
$\newcommand{\MJyr}{\MJ \mathrm{yr}^{-1}}$
$\newcommand{\Rd}{R_\mathrm{dust}}$
$\newcommand{\Rcav}{R_\mathrm{cav}}$
$\newcommand{\Rout}{R_\mathrm{out}}$
$\newcommand{\Rg}{R_\mathrm{gas}}$
$\newcommand{\micron}{\mu\mathrm{m}}$
$\newcommand{\Lacc}{L_\mathrm{acc}}$
$\newcommand{\LHa}{L_\mathrm{H\alpha}}$
$\newcommand{\FHa}{F_\mathrm{H\alpha}}$
$\newcommand{\AHa}{A_\mathrm{H\alpha}}$
$\newcommand{\Av}{A_\mathrm{V}}$
$\newcommand{\MaccAv}{\overline{\dot{M}_\mathrm{acc}}}$
$\newcommand{\chisquared}{\rchi^2}$
$\newcommand{\chimrs}{\rchi^2_\mathrm{MRS}}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.25}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Direct Measurement of Extinction in a Planet-Hosting Gap

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.26617-b31b1b.svg)](https://arxiv.org/abs/2509.26617)<mark>Appeared on: 2025-10-01</mark> -  _Accepted for publication in AJ. 22 pages, 10 figures, 5 tables_

</div>
<div id="authors">

G. Cugno, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Recent disk observations have revealed multiple indirect signatures of forming gas giant planets, but high-contrast imaging has rarely confirmed the presence of the suspected perturbers.Here, we exploit a unique opportunity provided by the background star AS 209bkg, which shines through a wide annular gap in the AS 209 disk, to perform transmission spectrophotometry and directly measure the extinction from gap material for the first time.By combining new VLT/SPHERE and JWST/NIRCam observations with archival HST data from 2005, we model the spectral energy distribution (SED) of AS 209bkg over a 19-year baseline. We find that the SED and its variability are best explained by increasing extinction along the line of sight as AS 209bkg approaches the gap edge in projection. The extinction is best described by a combination of ISM-like extinction component and a grey extinction component. This points to the presence of grains in the disk outer gap that are larger than in the ISM.We find that the extinction in the gap at $\lambda\sim4.0 \mu$ m is $A_{4 \mu\mathrm{m}} = 2.7^{+0.7}_{-0.7}$ mag, while at H $\alpha$ ( $\lambda=0.656 \mu$ m), where most searches for accretion signatures take place, the extinction could be as high as $A_\mathrm{H\alpha} = 4.2^{+0.9}_{-1.2}$ mag ( $A_V=4.6^{+1.0}_{-1.3}$ ). This suggests that even wide, deep gaps can significantly obscure emission from protoplanets, even those following a hot-start evolutionary model. Our extinction measurements help reconcile the discrepancy between ALMA-based predictions of planet-disk interactions and the non-detections from sensitive optical and near-infrared imaging campaigns.

</div>

<div id="div_fig1">

<img src="tmp_2509.26617/./Spectra_all.png" alt="Fig8" width="100%"/>

**Figure 8. -** SED fits for AS 209bkg using multiple extinction models. In each panel, the observed fluxes are reported in blue squares, while detection limits are shown in light blue arrows. The F300M detection limits, potentially impacted by the $3.0 \mu$m water-ice band, is shown for completeness but is not used in the fit. The best fit model is shown with a black line, while 100 random samples drawn by the posterior distribution are shown in grey. The log-evidence with respect to the `None' scenario (top left) is reported in each panel. (*fig:spectra_all*)

</div>
<div id="div_fig2">

<img src="tmp_2509.26617/./Proper_motion.png" alt="Fig7.1" width="50%"/><img src="tmp_2509.26617/./Proper_motion_AS209.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Astrometry of the AS 209 system. _ Left:_ Proper motion analysis of AS 209bkg. The red square represent the location of the source in 2005. The black line shows the expected trajectory from 2005 through 2024 assuming it is a stationary object, with the expected locations at the SPHERE and NIRCam epochs highlighted with black circles. The NIRCam and SPHERE data agree with the prediction (when considering the HST uncertainties), demonstrating that the source is indeed a background object. _ Right:_ Projected location of AS 209bkg in the AS 209 disk. On the right side, the millimeter image is reported, showcasing a series of dust rings  ([Andrews, Huang and Pérez 2018]()) . On the left side, the CO gas as measured by the DSHARP program  ([Andrews, Huang and Pérez 2018](), [Guzmán, Huang and Andrews 2018]())  is reported. AS 209bkg shines through the deep outer CO gap. Grey crosses mark the expected position of AS 209bkg in 2050 and 2075.  (*fig:proper_motion*)

</div>
<div id="div_fig3">

<img src="tmp_2509.26617/./Spectra_ep2005_all.png" alt="Fig9" width="100%"/>

**Figure 9. -** Same as Fig. \ref{fig:spectra_all}, with the addition of the red square representing the 2005 HST detection, and orange lines that represent the random sample and best-fit model for the 2005 epoch. In this case, the `Grey' model is used to benchmark the Bayesian log-evidences (*fig:spectra_ep2005*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.26617"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

9  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
